In [103]:
#!pip install gmplot
#!pip install pyproj
#!pip install mplstereonet
#!pip install shapely
#!pip install geopandas
#!pip install uncertainties
#!pip install plotly==4.3.0
#!pip install "notebook>=5.3" "ipywidgets>=7.2"
#!pip install statsmodels

In [104]:
def piano(A,B,C_,D):

    #calcolo E
    E=np.sqrt(A**2 + B**2 + 1)

    #calcolo i coseni
    cos_a = A/E

    cos_b = B/E

    cos_cp = C_/E  #gamma_p


    cos_c = np.cos((np.pi/2) + np.arccos(C_/E))   #controllare se è corretto 

    #calcolo la direzione teta del vettore, il coseni direttori danno la direzione del vettore occorrre aggiungere 90°
    teta_primo = np.arctan(cos_a/cos_b)   #espresso in radianti
    teta_deg = (teta_primo*180)/np.pi 

    #calcolo della direzione della massima pendenza sul piano
    #dip_direction = teta_deg 

    #calcolo l'immersione dip angolo tra la verticale e il piano
    dip_vert = np.arcsin(-1*cos_c)
    dip_deg = (dip_vert*180)/np.pi

    #calcolo l'immersione dip
    dip = dip_deg
    #occorre orientare correttamente il dip_direction
    #cond_1 = dip_direction = teta_deg
    #cond_2 = dip_direction = teta_deg + 180
    #cond_3 = dip_direction =  teta_deg + 360

    #def orientation(cos_a,cos_b):

    if cos_a > 0 and cos_b > 0:                
        dip_direction = teta_deg
        if cos_cp <0:
            dip_direction =  dip_direction + 180

    if  cos_a > 0 and cos_b < 0:
        dip_direction = teta_deg + 180
        if cos_cp <0:
            dip_direction =  dip_direction + 180


    if  cos_a < 0 and cos_b < 0:
        dip_direction = teta_deg + 180
        if cos_cp <0:
            dip_direction =  dip_direction - 180


    if  cos_a < 0 and cos_b > 0:
        dip_direction =  teta_deg + 360
        if cos_cp <0:
            dip_direction =  dip_direction - 180

    #creo l'array per il plotaggio su stereogramma
    plot_dip_direction = 0
    plot_dip = 0
    plot_dip_direction = np.append(plot_dip_direction, dip_direction)
    plot_dip = np.append(plot_dip, dip)
    print("dip_direction= %f ; dip= %f" % (dip_direction, dip))
    print(cos_a, cos_b)
    print('##################################################')
    print(teta_deg)
    output = "teta= %f ; dip= %f" % (dip_direction, dip)
    file = open("giacitura_cluster_xx_plane.txt","w")
    file.write(output)
    file.close()

    daframe_cluster_xx = pd.DataFrame.to_csv(df_cluster_2)
    file = open("daframe_cluster_xx_plane.csv","w")
    file.write(daframe_cluster_xx)
    file.close()

    #salva i risultati su di un file shape
    geometry = [Point(xy) for xy in zip(df_cluster_2.x , df_cluster_2.y)]
    df_shape = df_cluster_2.drop(['x', 'y','Ora', 'Data', 'Sec'], axis=1)
    crs = {'init': 'epsg:3003'}
    gdf = GeoDataFrame( df_shape, crs=crs, geometry=geometry)   #df_shape,
    gdf.to_file(driver = 'ESRI Shapefile', filename= "result_xx_plane.shp" )



    strike = plot_dip_direction - 90 
    dip =  plot_dip
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='stereonet')
    ax.plane(strike, dip)
    ax.pole(strike, dip ,markersize=5)
    ax.grid()
    plt.show()

In [105]:
def map_google():
    #Creo una mappa dei punti su google maps

    from gmplot import gmplot
    gmap = gmplot.GoogleMapPlotter( 44.5514, 9.4335, 13)

    # Scatter points
    top_attraction_lats = df['Latitud.']
    top_attraction_lons = df['Longit.']


    filter_attraction_lats = df_filter['Latitud.']
    filter_attraction_lons = df_filter['Longit.']

    cluster_attraction_lats = df_cluster_2['Latitud.']# *******ATTENZIONE SONO DATI FILTRATI E CLUSTERIZZATI
    cluster_attraction_lons = df_cluster_2['Longit.']



    gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=100, marker=False)

    #gmap.scatter(filter_attraction_lats, filter_attraction_lons, '#f90707', size=40, marker=False)

    gmap.scatter(cluster_attraction_lats, cluster_attraction_lons, '#1307f9', size=50, marker=False)

    gmap.heatmap(cluster_attraction_lats, cluster_attraction_lons ) # crea una heatmap
    # Draw
    gmap.draw("my_map_time.html")
    

In [106]:
def plot_3d_plane(Z, z_err1, z_err2):
    
    

    trace1 = go.Scatter3d(
    x=data_orig[:,0],
    y=data_orig[:,1],
    z=data_orig[:,2],
    mode='markers',
    marker=dict(size=4, color='red', line=dict(color='black', width=0.5), opacity=0.85)
)


    trace2 = go.Surface(z=Z, x=X, y=Y, colorscale='RdBu', opacity=0.6)
    trace3 = go.Surface(z=z_err_1, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace4 = go.Surface(z=z_err_2, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    #best_fit_plane = trace3
    # Package the trace dictionary into a data object
    data_test1 = go.Data([ trace1, trace2,trace3,trace4])

    # Dictionary of style options for all axes
    axis = dict(
        showbackground=True, # show axis background
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey
        gridcolor="rgb(255, 255, 255)",       # set grid line color
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
    )

    # Make a layout object
    layout = go.Layout(
        title='plane _error', # set plot title
        scene=go.Scene(  # axes are part of a 'scene' in 3d plots
            xaxis=go.XAxis(axis), # set x-axis style
            yaxis=go.YAxis(axis), # set y-axis style
            zaxis=go.ZAxis(axis)),  # set z-axis style
    )

    # Make a figure object
    fig = go.Figure(data=data_test1, layout=layout)

    # Send to Plotly and show in notebook
    #py.iplot(fig, filename='test1') # decommentare per plottare su server online
    plot(fig)
    iplot(fig)

In [107]:
def plot_9_plane(Z,Z_0,Z_1,Z_2,Z_3,Z_4,Z_5,Z_6,Z_7):
    
    

    trace1 = go.Scatter3d(
    x=data_orig[:,0],
    y=data_orig[:,1],
    z=data_orig[:,2],
    mode='markers',
    marker=dict(size=4, color='red', line=dict(color='black', width=0.5), opacity=0.85)
)


    trace2 = go.Surface(z=Z, x=X, y=Y, colorscale='RdBu', opacity=0.6)
    trace3 = go.Surface(z=Z_0, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace4 = go.Surface(z=Z_1, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace5 = go.Surface(z=Z_2, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace6 = go.Surface(z=Z_3, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace7 = go.Surface(z=Z_4, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace8 = go.Surface(z=Z_5, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace9 = go.Surface(z=Z_6, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    trace10 = go.Surface(z=Z_7, x=X, y=Y, colorscale='Viridis', opacity=0.6,showscale=False,)
    #best_fit_plane = trace3
    # Package the trace dictionary into a data object
    data_test1 = go.Data([ trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10])

    # Dictionary of style options for all axes
    axis = dict(
        showbackground=True, # show axis background
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey
        gridcolor="rgb(255, 255, 255)",       # set grid line color
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
    )

    # Make a layout object
    layout = go.Layout(
        title='plane _error', # set plot title
        scene=go.Scene(  # axes are part of a 'scene' in 3d plots
            xaxis=go.XAxis(axis), # set x-axis style
            yaxis=go.YAxis(axis), # set y-axis style
            zaxis=go.ZAxis(axis)),  # set z-axis style
    )

    # Make a figure object
    fig = go.Figure(data=data_test1, layout=layout)

    # Send to Plotly and show in notebook
    #py.iplot(fig, filename='test1') # decommentare per plottare su server online
    plot(fig)
    iplot(fig)

In [108]:
# IMPORTANTE ATTENZIONE OCCORRE CONVERTIRE IL FILE DIINGRESSO DEI PUNTI
# IN COORDINATE PIANE METRICHE 

import pandas as pd
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import datetime
from shapely.geometry import Point
from geopandas import GeoDataFrame


##################################error
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import uncertainties.umath as umath
import uncertainties.unumpy  as unumpy
import math


import mplstereonet
#from mpl_toolkits.basemap import pyproj
#from mpl_toolkits.basemap import Basemap
from pyproj import Proj, transform
###################################plotly_block###################################################### 
# Import plotly package online####################
import plotly
#plotly.tools.set_credentials_file(username='davide.schenone', api_key='WBSgVI0FXgSIRecK2cHU')
import plotly.graph_objs as go
# Check ploltly version
plotly.__version__

#plotly offline###############
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) #attivando o disattivando questa riga si ottiene un plot 
                                   #contestuale rispettivamente in notebook oppure su un html
import statsmodels.api as sm  #https://www.statsmodels.org/stable/regression.html
    
# To communicate with Plotly's server, sign in with credentials file
#import plotly.plotly as py
################################################################################################

from pandas import ExcelWriter
from pandas import ExcelFile
# Create data with x and y random over [-2, 2], and z a Gaussian function of x and y.
#np.random.seed(12345)
#x = 2 * (np.random.random(500) - 0.5)
#y = 2 * (np.random.random(500) - 0.5)

#def f(x, y):
 #   return np.exp(-(x + y ** 2))

#z = f(x, y)

#imposto i dati
#importa il file originale
#df = pd.read_csv('Saorge_Taggia_erh_erz_prof.csv', sep='\t', names=['y', 'x','z','a','b'])

#importa il file convertito con convergo
df = pd.read_excel('terremoti_santo.xlsx', sheet_name='Foglio1')


df['z'] = df['Prof corr']  #creo il campo z profondità con valori negativi


df['Data'] = df['Data'].astype(str).str.zfill(6) #riempie la mancanza dello zero nei primi valori
df['Data'] = pd.to_datetime(df['Data'], format='%y%m%d')
df['Data_subtr']=pd.to_datetime('890101', format='%y%m%d')
df['Deltatime'] = df['Data'] - df['Data_subtr']
df['Deltatime'] = df['Deltatime'].dt.days
#con questo posso fare dei cluster temporali

#converto le coordinate wgs 84 in UTM epsg 3003 Gauss-Boaga 
x1 = np.array(df['Longit.'])
y1 = np.array(df['Latitud.'])


inProj = Proj('epsg:4326')
outProj = Proj('epsg:3003')
x2,y2 = transform(inProj,outProj, x1, y1)

df['x'] = x2
df['y'] = y2

#con questo posso fare dei cluster temporali
#IMPOSTO I FILTRI

#df_filter = df.query('Erh<1 and Erz<4') #imposto i parametri di filtro sugli errori orizzontali e verticali


#df_filter = df.query('<Deltatime<') #imposto il filtro sulla data

#imposto i pesi per il calcolo del piano da utilizzare in caso di regressione lineare pesata

df['weight_calc'] = df['Nf']/24 - 0.25
def weight(df):
    if df['Nf'] <=6 :
        return 0
    
    elif  df['Nf'] > 6 and df['Nf']  < 30   :
        return df['weight_calc']
            
    elif  df['Nf'] >= 30 :
        return 1

df['weights'] =  df.apply(weight, axis=1) 
 

    
df_filter = df


# Clustering

In [109]:
coords =np.vstack(df_filter["Deltatime"])

from sklearn.cluster import DBSCAN
import numpy as np
#standard eps=70 min_samples = 10
db = DBSCAN(eps=70, min_samples=10).fit(coords)
labels = db.labels_
from collections import Counter


df_filter = df_filter.assign(Classe = labels)
print(Counter(labels))
#filtro la label di interesse
df_cluster = df_filter[df_filter['Classe'] == 0] # <=========================

#################################################
df_filter_2 = df_cluster
coords_2 = df_filter_2[['x', 'y','z']].to_numpy() #previus as_matrix

from sklearn.cluster import DBSCAN
import numpy as np
#base eps 3000 min_samples 5
db = DBSCAN(eps=3000, min_samples=5).fit(coords_2)
labels_2 = db.labels_
from collections import Counter


df_filter_2 = df_filter_2.assign(Classe_spaziale = labels_2)
print(Counter(labels_2))

#filtro la label di interesse
df_cluster_2 = df_filter_2[df_filter_2['Classe_spaziale'] == 0]  # <=========================

Counter({0: 62, -1: 61, 3: 42, 2: 25, 1: 19})
Counter({0: 46, -1: 10, 1: 6})


# imposto i dati 

In [110]:
#importo il file depurato degli outlayer:
#df = pd.read_csv('taggia_saorge_ripulita_outlier.csv', sep=',', names=['x', 'y','z'])

x = df_cluster_2['x']  

y = df_cluster_2['y']   

z = df_cluster_2['z']

W = df_cluster_2['weights']


data_orig =  df_cluster_2[['x','y','z']].to_numpy()

# Minimi quadrati

In [111]:

A_ = np.c_[data_orig[:,0], data_orig[:,1], np.ones(data_orig.shape[0])]
L = data_orig[:,2]

wls_model = sm.WLS(L,A_,W)
wls_result = wls_model.fit()

print(wls_result.summary())
print(wls_result.conf_int())


x1,x2,const= wls_result.params
# x1_err, x2_err, const_err = wls_result.conf_int() prima versione prendeva in considerazione intervallo di confidenza
x1_err, x2_err, const_err = wls_result.bse


x1_err_max = x1 + x1_err
x2_err_max = x2 + x2_err
const_err_max = const + const_err


x1_err_min = x1 - x1_err
x2_err_min = x2 -  x2_err
const_err_min = const - const_err

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.416
Model:                            WLS   Adj. R-squared:                  0.388
Method:                 Least Squares   F-statistic:                     15.29
Date:                Fri, 24 Apr 2020   Prob (F-statistic):           9.65e-06
Time:                        22:20:05   Log-Likelihood:                -424.58
No. Observations:                  46   AIC:                             855.2
Df Residuals:                      43   BIC:                             860.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7266      0.211      3.437      0.0

In [112]:
print(wls_result.bse)

[2.11391517e-01 1.87679276e-01 1.37335043e+06]


In [114]:
A=x1
B=x2
D=const
C_ = -1


A = ufloat(A, 2.32031598e-01)  # x = 1+/-0.1
B = ufloat(B, 2.45795013e-01)
D = ufloat(D, 1.45839563e+06)
C_= ufloat(C_, 0)





    #calcolo E
E=umath.sqrt(A**2 + B**2 + 1)

#calcolo i coseni
cos_a = A/E

cos_b = B/E

cos_cp = C_/E  #gamma_p


cos_c = unumpy.cos((math.pi/2) + unumpy.arccos(C_/E))   #controllare se è corretto 

#calcolo la direzione teta del vettore, il coseni direttori danno la direzione del vettore occorrre aggiungere 90°
teta_primo = unumpy.arctan(cos_a/cos_b)   #espresso in radianti
teta_deg = (teta_primo*180)/np.pi 

#calcolo della direzione della massima pendenza sul piano
#dip_direction = teta_deg 

#calcolo l'immersione dip angolo tra la verticale e il piano
dip_vert = unumpy.arcsin(-1*cos_c)
dip_deg = (dip_vert*180)/np.pi

#calcolo l'immersione dip
dip = dip_deg
#occorre orientare correttamente il dip_direction
#cond_1 = dip_direction = teta_deg
#cond_2 = dip_direction = teta_deg + 180
#cond_3 = dip_direction =  teta_deg + 360

#def orientation(cos_a,cos_b):

if cos_a > 0 and cos_b > 0:                
    dip_direction = teta_deg
    if cos_cp <0:
        dip_direction =  dip_direction + 180

if  cos_a > 0 and cos_b < 0:
    dip_direction = teta_deg + 180
    if cos_cp <0:
        dip_direction =  dip_direction + 180


if  cos_a < 0 and cos_b < 0:
    dip_direction = teta_deg + 180
    if cos_cp <0:
        dip_direction =  dip_direction - 180


if  cos_a < 0 and cos_b > 0:
    dip_direction =  teta_deg + 360
    if cos_cp <0:
        dip_direction =  dip_direction - 180

#creo l'array per il plotaggio su stereogramma
plot_dip_direction = 0
plot_dip = 0
plot_dip_direction = np.append(plot_dip_direction, dip_direction)
plot_dip = np.append(plot_dip, dip)
print("dip_direction= %f ; dip= %f" % (dip_direction, dip))
print(cos_a, cos_b)
print('##################################################')
print(teta_deg)
output = "teta= %f ; dip= %f" % (dip_direction, dip)
file = open("giacitura_cluster_xx_plane.txt","w")
file.write(output)
file.close()

daframe_cluster_xx = pd.DataFrame.to_csv(df_cluster_2)
file = open("daframe_cluster_xx_plane.csv","w")
file.write(daframe_cluster_xx)
file.close()

#salva i risultati su di un file shape
geometry = [Point(xy) for xy in zip(df_cluster_2.x , df_cluster_2.y)]
df_shape = df_cluster_2.drop(['x', 'y','Ora', 'Data', 'Sec'], axis=1)
crs = {'init': 'epsg:3003'}
gdf = GeoDataFrame( df_shape, crs=crs, geometry=geometry)   #df_shape,
gdf.to_file(driver = 'ESRI Shapefile', filename= "result_xx_plane.shp" )



strike = plot_dip_direction - 90 
dip =  plot_dip
fig = plt.figure()
ax = fig.add_subplot(111, projection='stereonet')
ax.plane(strike, dip)
ax.pole(strike, dip ,markersize=5)
ax.grid()
plt.show() 


print(dip)
print(dip_direction)

TypeError: can't convert an affine function (<class 'uncertainties.core.AffineScalarFunc'>) to float; use x.nominal_value

In [ ]:
A=x1
B=x2
C_=-1
D=const

print(A,B,C_,D)
piano(A,B,C_,D)

In [ ]:
A=x1
B=x2
C_=-1
D=const


piano(A,B,C_,D)
print(A,B,C_,D)

In [ ]:
A=x1_err_max
B=x2_err_max
C_=-1
D=const_err_max

print(A,B,C_,D)
piano(A,B,C_,D)

In [ ]:
A=x1_err_min
B=x2_err_min
C_=-1
D=const_err_min

print(A,B,C_,D)
piano(A,B,C_,D)

# Definisco le Z

In [ ]:
mn = np.min(data_orig, axis=0)
mx = np.max(data_orig, axis=0)
X,Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace( mx[1],mn[1], 20))


z_err_1 = x1_err_max*X + x2_err_max*Y + const_err_max
z_err_2 = x1_err_min*X + x2_err_min*Y + const_err_min


Z= wls_result.params[0]*X + wls_result.params[1]*Y + wls_result.params[2]

In [ ]:
mn

In [ ]:
wls_result.params[0]

In [ ]:
plot_3d_plane(Z)

In [ ]:
# creo una matrice dei coefficenti del piano attraverso la combinazione dei dell'errore dei coeficienti
import itertools
coef_matrix_err = []
for x in itertools.product(["A+", "A-"], ["B+",  "B-"],["C+","C-"]):
    coef_matrix_err.append(x) 
print(coef_matrix_err)

In [ ]:
# creo una matrice dei coefficenti del piano attraverso la combinazione dei dell'errore dei coeficienti

import itertools
coef_matrix_err = []
for x in itertools.product([x1_err_max, x1_err_min], [x2_err_max,  x2_err_min],[const_err_max,const_err_min]):
    coef_matrix_err.append(x) 
print(coef_matrix_err)

In [ ]:
A_ = globals()
B_ = globals()
D_ = globals()
Z_ = globals()

for x in range(0,8):
    A_['A_%d' % x],B_['B_%d' % x],D_['D_%d' % x] = coef_matrix_err[x]
    
    Z_['Z_%d' % x] = A_['A_%d' % x]*X + B_['B_%d' % x]*Y + D_['D_%d' % x]

In [ ]:
plot_9_plane(Z,Z_0,Z_1,Z_2,Z_3,Z_4,Z_5,Z_6,Z_7)